## Trabalho 4 - O problema do Caixeiro Viajante
* Criar um programa para resolver este problema usando a metodologia do caixeiro viajante por AGs.
* O algoritmo deverá buscar a melhor rota priorizando a minimização da distância percorrida.

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import random
from collections import defaultdict

from ipywidgets import *
import ipywidgets as widgets

import xlrd

from IPython.display import display, HTML

class Individuo():   
    
    def __init__(self, distancias, dicionario_cidades, geracao=1, **kwargs):
        self.distancias = distancias
        self.dicionario_cidades = dicionario_cidades        
        #Recuperando o idx da cidade de partida
        self.idx_cidade_de_partida = list(dicionario_cidades.keys())[list(dicionario_cidades.values()).index(cidade_de_partida.result)]        
        if geracao == 1:
            self.gera_cromossomo()
        self.geracao = geracao
        self.aptidao = 0
                        
    def __repr__(self): # formata representacao do objeto quanto utiliza print/display
        return np.array2string(self.cromossomo)
    
    def gera_cromossomo(self):
        
        self.cromossomo = np.arange(1,21) # de 1 a 20             
        
        # remove a cidade escolhida do cromossomo
        index_cidade_de_partida = 0
        self.cromossomo = np.delete(self.cromossomo, (self.idx_cidade_de_partida-1))
        
        np.random.shuffle(self.cromossomo)        
                
    def avaliacao(self):                  
        soma_distancias = 0
        
        # calcula distancias
        soma_distancias += calcula_distancia(self.idx_cidade_de_partida, self.cromossomo[0], self.distancias)
        for i in range(0,18):        
            if i != 18:
                soma_distancias += calcula_distancia(self.cromossomo[i], self.cromossomo[i+1], self.distancias)
        soma_distancias += calcula_distancia(self.cromossomo[18], self.idx_cidade_de_partida, self.distancias)
    
        self.aptidao = 1/soma_distancias
        
    def cruzamento(self, outro_individuo, distancias, dicionario_cidades):
        
        # o primeiro pai é copiado até o ponto de cruzamento
        f1_cromossomo = self.cromossomo[0:10]        
        
        # o segundo pai é rastreado e se o número ainda não estiver no filho, ele é adicionado
        for c in outro_individuo.cromossomo:
            if c not in f1_cromossomo:
                f1_cromossomo = np.concatenate((f1_cromossomo, [c]))
        
        f2_cromossomo = outro_individuo.cromossomo[0:10]
        for c in self.cromossomo:
            if c not in f2_cromossomo:
                f2_cromossomo = np.concatenate((f2_cromossomo, [c]))
            
        filhos = [f1_cromossomo, f2_cromossomo]
        
        filhos_individuos = [Individuo(distancias, dicionario_cidades, geracao=self.geracao+1),
                 Individuo(distancias, dicionario_cidades, geracao=self.geracao+1)]
        filhos_individuos[0].cromossomo = filhos[0]
        filhos_individuos[1].cromossomo = filhos[1]
        
        return filhos_individuos 
    
    def mutacao(self, taxa_mutacao):
        if random.random() < taxa_mutacao:                                     
            idxs = random.sample(range(0,19), 2)
            aux = self.cromossomo[idxs[0]]
            self.cromossomo[idxs[0]] = self.cromossomo[idxs[1]]
            self.cromossomo[idxs[1]] = aux
        return self
    
class AlgoritmoGenetico():
    
    def __init__(self, tam_populacao, taxa_cruzamento, taxa_mutacao,
                qtd_geracoes, selecao, elitismo, distancias, tam_torneio=0, **kwargs):
        self.tam_populacao = tam_populacao
        self.populacao = np.array([])
        self.taxa_cruzamento = taxa_cruzamento
        self.taxa_mutacao = taxa_mutacao
        self.qtd_geracoes = qtd_geracoes
        self.geracao = 0
        self.selecao = selecao
        self.tam_torneio = tam_torneio
        self.elitismo = elitismo
        self.lista_solucao = []
        self.melhor_solucao = 0
        self.distancias = distancias
        self.dicionario_cidades = []
    
    def inicializa_populacao(self):   
        lista = []
        for i in range(self.tam_populacao):
            lista.append(Individuo(self.distancias, self.dicionario_cidades))
        self.populacao = np.array(lista)
        self.melhor_solucao = self.populacao[0]    
        
    def ordena_populacao(self):
        self.populacao = sorted(self.populacao,
                               key =  lambda individuo: individuo.aptidao,
                               reverse = True)
        self.populacao = np.array(self.populacao)
        
    def calcula_aptidao(self):
        for ind in self.populacao:
            ind.avaliacao()
    
    def melhor_individuo(self, individuo):
        if individuo.aptidao > self.melhor_solucao.aptidao:
            self.melhor_solucao = individuo
    
    def soma_aptidoes(self):
        soma = 0
        for i in self.populacao:
            soma += i.aptidao
        return soma
            
    def seleciona_pais(self, soma_aptidoes=0):
        '''
        retorna um array com o par de pais com base no tipo de selecao
        '''
        if self.selecao == 'roleta':
            r_pais = [random.random() * soma_aptidoes]
            while True:
                r = random.random() * soma_aptidoes
                if r not in r_pais:
                    r_pais.append(r)
                    break
                    
            pais = []
            for r in r_pais:
                pai = -1
                i = 0
                soma = 0
                while i < len(self.populacao) and soma < r:
                    soma += self.populacao[i].aptidao
                    pai += 1
                    i += 1
                pais.append(self.populacao[pai])
            return pais
        
        elif self.selecao == 'torneio':
            indices_torneio = random.sample(range(0, self.tam_populacao), self.tam_torneio)
            torneio = self.populacao[indices_torneio]
            torneio = sorted(torneio, key=lambda i: i.aptidao, reverse=True)
            return [torneio[0], torneio[1]]
        
    
    def seleciona_elitismo(self):
        '''
        retorna lista contendo os individuos elitistas
        '''
        self.ordena_populacao()        
        return self.populacao[0:self.elitismo]

########## entradas dos parametros ##########

print('\n\nTamanho da população:')
def f(populacao=1000):
    return populacao
populacao = interactive(f, populacao=(10, 1000, 10))
display(populacao)

print('\n\nProbabilidade de cruzamento:')
def f(cruzamento=95.0):
    return cruzamento
cruzamento = interactive(f, cruzamento=(5.0, 95.0, 0.5))
display(cruzamento)

print('\n\nProbabilidade de mutação:')
def f(mutacao=0.1):
    return mutacao
mutacao = interactive(f, mutacao=(1.0, 10.0, 0.5))
display(mutacao)

print('\n\nQuantidade de gerações:')
def f(geracoes=30):
    return geracoes
geracoes = interactive(f, geracoes=(10, 1000))
display(geracoes)

print('\n\nMétodo de seleção:')
def f(selecao):
    return selecao
selecao = interactive(f, selecao=['Torneio', 'Roleta'])
display(selecao)

print("""\n\nTamanho do Torneio:\n *ignorar caso o 'Método de seleção' selecionado seja diferente de 'Torneio'*""")
def f(tam_torneio=10):
    return tam_torneio
tam_torneio = interactive(f, tam_torneio=(2, 50, 2))
display(tam_torneio)

print('\n\nTamanho do elitismo:')
def f(elitismo=5):
    return elitismo
elitismo = interactive(f, elitismo=(0, 10))
display(elitismo)

print('\n\nCidade de Partida:')
def f(cidade_de_partida):
    return cidade_de_partida
cidade_de_partida = interactive(f, cidade_de_partida=['Uberaba','Uberlândia','Araxá','Patos de Minas','Patrocínio','Monte Carmelo','Araguari','Ituiutaba','Prata','Frutal','Conceição das Alagoas','Campo Florido','Perdizes','Santa Juliana','Nova Ponte','Delta','Água Comprida','Sacramento','Conquista','Comendador Gomes'])
display(cidade_de_partida)

def xlread(arq_xls):

    # le o arquivo xlsx
    xlsx = xlrd.open_workbook(arq_xls)
    sheet_names = xlsx.sheet_names()
    sheet = xlsx.sheet_by_name(sheet_names[0])                 
    
    # monta a matrix com as distancias de todas as cidades
    distancias = []
    for nrow in range(1,sheet.nrows):
        row = []
        for ncol in range(0,sheet.ncols):
            row.append(sheet.cell_value(nrow, ncol)) 
        distancias.append(row)          
    distancias = np.array(distancias)    
        
    return distancias    

def calcula_distancia(cidade1, cidade2, distancias):     
    '''
    cidade1: indice da cidade retornada do dicionario de cidade - 1 para equivaler ao indice da tabela
    cidade2: indice da cidade retornada - como retorna direto do dicionario e não pode-se considerar a coluna 1, não precisa alterar
    '''    
    distancia = 0
    distancia = distancias[cidade1-1][cidade2]             
    return float(distancia)
    
########## processamento ##########
def processar():        
    
    alg = AlgoritmoGenetico(
        tam_populacao = populacao.result,
        taxa_cruzamento = cruzamento.result/100.0,
        taxa_mutacao = mutacao.result/100.0,
        qtd_geracoes = geracoes.result,
        selecao = selecao.result.lower(),
        elitismo = elitismo.result,
        distancias = xlread("distances.xlsx"),
        tam_torneio = tam_torneio.result)
    
    ## inicio ##
    
    # pega o nome de todas as cidades existentes na planilha
    cidades = []
    for i in range(0,len(alg.distancias),1):
        cidades.append(alg.distancias[i][0])
    
    # monta um dicionario com os nomes das cidades {1: 'Uberaba'...}
    keys = np.arange(1,21)    
    dicionario_cidades = {k:c for k, c in zip(keys, cidades)} 
    alg.dicionario_cidades = dicionario_cidades
    
    alg.inicializa_populacao()
    
    graf_aptidoes = []
    graf_geracoes = []    
    
    for g in range(0, alg.qtd_geracoes):
        graf_geracoes.append(g)        
        
        alg.calcula_aptidao()
        alg.ordena_populacao()
        
        nova_popu = []
        soma_apt = alg.soma_aptidoes() if alg.selecao == 'roleta' else 0
        for c in range(0, alg.tam_populacao, 2):
            pais = alg.seleciona_pais(soma_apt)            
            if random.random() <= alg.taxa_cruzamento:
                filhos = pais[0].cruzamento(pais[1], alg.distancias, alg.dicionario_cidades)
                for filho in filhos:                                        
                    nova_popu.append(filho.mutacao(alg.taxa_mutacao))
            else:
                for pai in pais:
                    nova_popu.append(pai)
                            
        elitistas = alg.seleciona_elitismo()                
        indices = random.sample(range(0, alg.tam_populacao), len(elitistas)) # escolhe de 0 a 50, um número

        for idx in range(0, len(elitistas)):
            alg.populacao[indices[idx]] = elitistas[idx]
                
        alg.populacao = np.array(nova_popu)
        
        alg.calcula_aptidao()        
        alg.ordena_populacao()
        alg.lista_solucao.append(alg.populacao[0])
        alg.melhor_individuo(alg.populacao[0])
        
        graf_aptidoes.append(1/alg.populacao[0].aptidao)        
          
    string = "<h4><strong>Geração da melhor solução:</strong> {:}</h4>".format(alg.melhor_solucao.geracao)
    display(HTML(string))
    
    string = "<h4><strong>Melhor solução:</strong></h4>"
    display(HTML(string))
    
    print('{0}'.format(dicionario_cidades[list(dicionario_cidades.keys())[list(dicionario_cidades.values()).index(cidade_de_partida.result)]]))
    for i in alg.melhor_solucao.cromossomo:
        print('{0}'.format(dicionario_cidades[i]))
    print('{0}'.format(dicionario_cidades[list(dicionario_cidades.keys())[list(dicionario_cidades.values()).index(cidade_de_partida.result)]]))
    
    string = "<h4><strong>Distância total:</strong> {:}</h4>".format(1/alg.melhor_solucao.aptidao)
    display(HTML(string))
    
    N = 1000
    random_x = np.random.randn(N)
    random_y = np.random.randn(N)
    
    l = plt.plot(graf_geracoes, graf_aptidoes, 'ro')
    plt.setp(l, markersize=15)
    plt.setp(l, markerfacecolor='C0')

    string = "<h4><strong>Gráfico das aptidões:</strong></h4>"
    display(HTML(string))    
    plt.show()
    
widgets.interact_manual.opts['manual_name'] = 'Processar algoritmo' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao



Tamanho da população:


interactive(children=(IntSlider(value=1000, description='populacao', max=1000, min=10, step=10), Output()), _d…



Probabilidade de cruzamento:


interactive(children=(FloatSlider(value=95.0, description='cruzamento', max=95.0, min=5.0, step=0.5), Output()…



Probabilidade de mutação:


...



Tamanho do elitismo:


interactive(children=(IntSlider(value=5, description='elitismo', max=10), Output()), _dom_classes=('widget-int…



Cidade de Partida:


interactive(children=(Dropdown(description='cidade_de_partida', options=('Uberaba', 'Uberlândia', 'Araxá', 'Pa…

interactive(children=(Button(description='Processar algoritmo', style=ButtonStyle()), Output()), _dom_classes=…